# Understanding the data
# Description
This project is to build a model that predicts the human activities such as Walking, Walking_Upstairs, Walking_Downstairs, Sitting, Standing and Laying.

This dataset is collected from 30 persons(referred as subjects in this dataset), performing different activities with a smartphone to their waists. The data is recorded with the help of sensors Accelerometer and Gyroscope in that smartphone. 


# Importing Necessary Packages


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline


# Loading the Train and Test Data


In [ ]:
train=pd.read_csv("../input/human-activity-recognition-with-smartphones/train.csv")

In [ ]:
train.head()

In [ ]:
test=pd.read_csv("../input/human-activity-recognition-with-smartphones/test.csv")

In [ ]:
test.head()

In [ ]:
train.shape                                   # checking the number of rows and columns

In [ ]:
test.shape                                     # checking the number of rows and columns

In [ ]:
train.isnull().sum().any()                     # checking the number of missing values

In [ ]:
test.isnull().sum().any()                       # checking the number of missing values

In [ ]:

#Observation: There are no missing values in train and test data

In [ ]:
train.dtypes.value_counts()                      # checking the datatype of the columns

In [ ]:
test.dtypes.value_counts()                          # checking the datatype of the columns

In [ ]:
train['Activity'].value_counts()                  # Checking the number of records for each labels


# Exploratory Analysis


In [ ]:

#How active are the participants

In [ ]:
# we are going to count how many sensor measurments are there for each activity for each partcipant

pivoted=train.pivot_table(index="subject",columns="Activity",aggfunc='count').iloc[:,:6]
count_df=pd.DataFrame(pivoted.to_records())        #convert pivot table to dataframe
count_df=count_df.set_index ("subject")         # Change the index
count_df.columns =["LAYING","SITTING","STANDING","WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"]
count_df

In [ ]:
# SAMPLING IS DONE AT 1.28 SECONDS,SO THE TOTAL DURATION FOR ALL THE ACTIONS WILL BE COUNT 8 1.28
# THIS IS A SAMPLE ,WHICH IS WHY THE DURATION IS QUIET LESS.IF A PERSON WEARS THE TRACKER FOR THE ACTIVITIES
# YOU CAN MEASURE HOW MANY HOURS HE WALKS,SLEEPS,SITS..ETC..

duration_df=count_df * 1.28
duration_df

In [ ]:
# Feature Engineering : Creating new columns,active and passive
duration_df['active']= duration_df['WALKING']+duration_df['WALKING_UPSTAIRS']+duration_df['WALKING_DOWNSTAIRS']
duration_df['passive']=duration_df['LAYING']+duration_df['SITTING']+duration_df['STANDING']

In [ ]:
duration_df

In [ ]:
plt.figure(figsize=(14,4))
sns.barplot(x=duration_df.index,y=duration_df.LAYING)

In [ ]:

#Observation: Person 1 and 14 sleep very little
#The missing numbers in x-axis are the people who are present in test data

In [ ]:
x=duration_df.index
plt.figure(figsize=(14,4))
ax=plt.subplot(111)
ax.bar(x-0.4, duration_df.active, width=0.4, color='b', align='center')
ax.bar(x, duration_df.passive, width=0.4, color='g', align='center')
plt.show()

In [ ]:
#Observation: Person 19 and 21 are highly inactive

In [ ]:
plt.figure(figsize=(14,4))
sns.barplot(x=duration_df.index,y=duration_df.WALKING)

In [ ]:

#Observation: person 1 and 25 are walking a lot

# MODEL BUILDING


In [ ]:
x_train=train.drop(['Activity','subject'],axis=1)
y_train=train['Activity']

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_test=test.drop(['Activity','subject'],axis=1)
y_test=test['Activity']

In [ ]:
# lets reduce the dimensions from 561 columns to 2 columns
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
principalComponents=pca.fit_transform(x_train)
principalDf = pd.DataFrame(principalComponents,columns=['pc1','pc2'])

In [ ]:
principalDf.head()

In [ ]:
finalDf=pd.concat([principalDf,y_train],axis=1)

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,8))
sns.scatterplot(x='pc1',y='pc2',data=principalDf,hue=y_train)

In [ ]:
#Observations: The resting and ative states are clearly separable here 
#We can differentiate walking,walking-upstairs and walking-downstairs. 
#There is not much differentiation between sitting,laying and standing

# EDA for 3D plot

In [ ]:
from sklearn .decomposition import PCA
pca = PCA(n_components=3)
principalComponents=pca.fit_transform(x_train)
principalDf=pd.DataFrame(principalComponents,columns=['pc1','pc2','pc3'])

In [ ]:
principalDf.head()

In [ ]:
finalDf=pd.concat([principalDf,y_train],axis=1)

In [ ]:
import plotly.express as px
fig=px.scatter_3d(finalDf,x="pc1",y="pc2",z="pc3",color="Activity",color_discrete_map={"pc1": "pc2","pc3":"green"})
fig.show()

In [ ]:

#Observations: Here we are able to see all the different activities using 3 principal components.

# Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_clf=DecisionTreeClassifier()

In [ ]:
dt_clf.fit(x_train,y_train)               #fit the model in train 

In [ ]:
y_pred=dt_clf.predict(x_test)           #make predictions on test data

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
cm=confusion_matrix(y_pred,y_test)

In [ ]:
print(cm)

In [ ]:
cr=classification_report(y_pred,y_test)
print(cr)

In [ ]:
print(accuracy_score(y_pred,y_test))

# Conclusion

In [ ]:
#Human  activity  recognition  has  broad  applications  in medical research and human survey system. 
#In this project, we  designed  a  smartphone-based  recognition  system  that recognizes five human activities: walking, limping, jogging, going upstairs and going downstairs. 
#The activity data were trained and tested using  PCA , Decision Tree algorithm and Confusion Matrix . 
#The best  classification  rate  in our experiment  accuracy_score was 0.8646080760095012
